In [1]:
import deep_predictor
from collections.abc import Mapping
import torch
import scanpy as sc

Global seed set to 0
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import numpy as np
import pandas as pd

In [3]:
adata = sc.read_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/adata_segment_subset.h5ad")

In [7]:
Time = []
for i in adata.obs["stage"] :
    Time.append(float(i.split("E")[1]))

In [8]:
adata.obs["Time"] = np.array(Time)

In [9]:
Ectoderm = [27,30,31,32,33,40]
Mesoderm = [13,15,16,19,14,35,36,41]

In [10]:
#Training Data
segments = [27,30,31,32,33]
training_data_ecto = adata[adata.obs['segment'] == 40]
for segment in segments:
    segment_data = adata[adata.obs['segment'] == segment]
    training_data_ecto = training_data_ecto.concatenate(segment_data)
training_data_ecto.obs["cells"] = "Training data"

KeyError: 'segment'

In [ ]:
#Training Data
segments = [13,15,16,19,14,35,36]
training_data_meso = adata[adata.obs['segment'] == 41]
for segment in segments:
    segment_data = adata[adata.obs['segment'] == segment]
    training_data_meso = training_data_meso.concatenate(segment_data)
training_data_meso.obs["cells"] = "Training data"

In [11]:
Mesoderm_lineage = training_data_meso[(training_data_meso.obs["Time"] <= 7.5) | (training_data_meso.obs["Time"] == 8.5)]

NameError: name 'training_data_meso' is not defined

In [23]:
Ectoderm_lineage = training_data_ecto[(training_data_ecto.obs["Time"] <= 7.5) | (training_data_ecto.obs["Time"] == 8.5)]

In [24]:
## Removing the incorrectly assigned cells from both lineages
## Mesoderm

In [25]:
celltypes_to_remove = ['Def. endoderm', 'Gut', 'PGC','Rostral neurectoderm', 'Surface ectoderm']

In [26]:
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(Mesoderm_lineage[Mesoderm_lineage.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(Mesoderm_lineage.obs_names),cells_to_remove)

In [27]:
Mesoderm_lineage = Mesoderm_lineage[cells_to_keep]

In [28]:
Mesoderm_lineage.obs["celltype"].value_counts()

Somitic mesoderm             533
Nascent mesoderm             480
Cardiomyocytes               437
Mixed mesoderm               407
Mesenchyme                   392
Intermediate mesoderm        275
Paraxial mesoderm            217
Pharyngeal mesoderm          166
Caudal epiblast               65
Primitive Streak              51
Caudal Mesoderm               17
Anterior Primitive Streak      8
Name: celltype, dtype: int64

In [29]:
## Ectoderm

In [30]:
celltypes_to_remove = ['Nascent mesoderm','PGC','Intermediate mesoderm','Caudal Mesoderm']

In [31]:
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(Ectoderm_lineage[Ectoderm_lineage.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(Ectoderm_lineage.obs_names),cells_to_remove)

In [32]:
Ectoderm_lineage = Ectoderm_lineage[cells_to_keep]

In [33]:
Ectoderm_lineage.obs["celltype"].value_counts()

Neural crest                    536
Forebrain/Midbrain/Hindbrain    408
Surface ectoderm                174
NMP                             150
Spinal cord                     136
Epiblast                        122
Primitive Streak                 71
Rostral neurectoderm             56
Caudal epiblast                  30
Caudal neurectoderm               8
Name: celltype, dtype: int64

In [34]:
prog = Ectoderm_lineage[Ectoderm_lineage.obs["Time"] <= 7.5]
mature = Ectoderm_lineage[Ectoderm_lineage.obs["Time"] == 8.5]

In [35]:
prog.obs["celltype"].value_counts()

Epiblast                122
Primitive Streak         71
Rostral neurectoderm     51
Caudal epiblast          30
Surface ectoderm         16
Caudal neurectoderm       8
Name: celltype, dtype: int64

In [36]:
mature.obs["celltype"].value_counts()

Neural crest                    536
Forebrain/Midbrain/Hindbrain    408
Surface ectoderm                158
NMP                             150
Spinal cord                     136
Rostral neurectoderm              5
Name: celltype, dtype: int64

In [41]:
## Divide Ectoderm into progenitors and mature cell types
## And remove the unwanted cells placed in these categories

# mesoderm

In [37]:
prog = Mesoderm_lineage[Mesoderm_lineage.obs["Time"] <= 7.5]
mature = Mesoderm_lineage[Mesoderm_lineage.obs["Time"] == 8.5]

In [43]:
celltypes_to_remove = ['Mesenchyme','Pharyngeal mesoderm','Paraxial mesoderm','Somitic mesoderm']
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(prog[prog.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(prog.obs_names),cells_to_remove)
prog = prog[cells_to_keep]

In [63]:
celltypes_to_remove = ['Intermediate mesoderm','Caudal Mesoderm']
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(mature[mature.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(mature.obs_names),cells_to_remove)
mature = mature[cells_to_keep]

In [56]:
# Starting data 
progenitors_data = prog[prog.obs['celltype'] == "Nascent mesoderm"]
# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),100)
downsampled_proge_data = progenitors_data[random_index]

segments = ['Mixed mesoderm','Caudal epiblast','Primitive Streak','Intermediate mesoderm','Anterior Primitive Streak','Caudal Mesoderm']
for segment in segments:
    cell_data = prog[prog.obs['celltype'] == segment]
    random_index = np.random.choice(np.arange(0,np.shape(cell_data)[0]),100)
    cell_data = cell_data[random_index]
    downsampled_proge_data = downsampled_proge_data.concatenate(cell_data)
    

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [70]:
# Starting data 
mature_data = mature[mature.obs['celltype'] == 'Cardiomyocytes']
# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(mature_data)[0]),150)
downsampled_mature_data = mature_data[random_index]

segments = ['Mesenchyme', 'Paraxial mesoderm','Pharyngeal mesoderm', 'Somitic mesoderm']
for segment in segments:
    cell_data = mature[mature.obs['celltype'] == segment]
    random_index = np.random.choice(np.arange(0,np.shape(cell_data)[0]),150)
    cell_data = cell_data[random_index]
    downsampled_mature_data = downsampled_mature_data.concatenate(cell_data)
    

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [79]:
downsampled_mesoderm = downsampled_proge_data.concatenate(downsampled_mature_data)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


# ectoderm

In [97]:
prog = Ectoderm_lineage[Ectoderm_lineage.obs["Time"] <= 7.5]
mature = Ectoderm_lineage[Ectoderm_lineage.obs["Time"] == 8.5]

In [102]:
prog.obs["celltype"].value_counts()

Epiblast                122
Primitive Streak         71
Rostral neurectoderm     51
Caudal neurectoderm       8
Name: celltype, dtype: int64

In [103]:
mature.obs["celltype"].value_counts()

Neural crest                    536
Forebrain/Midbrain/Hindbrain    408
Surface ectoderm                158
NMP                             150
Spinal cord                     136
Name: celltype, dtype: int64

In [100]:
celltypes_to_remove = ['Caudal epiblast','Surface ectoderm']
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(prog[prog.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(prog.obs_names),cells_to_remove)
prog = prog[cells_to_keep]

In [101]:
celltypes_to_remove = ['Intermediate mesoderm','Rostral neurectoderm']
cells_to_remove = []
for i in celltypes_to_remove : 
    cells_to_remove.append(np.array(mature[mature.obs["celltype"] == i].obs_names))
cells_to_remove = np.concatenate(cells_to_remove)    
cells_to_keep = np.setdiff1d(np.array(mature.obs_names),cells_to_remove)
mature = mature[cells_to_keep]

In [104]:
# Starting data 
progenitors_data = prog[prog.obs['celltype'] == 'Epiblast']
# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),100)
downsampled_proge_data = progenitors_data[random_index]

segments = ['Caudal neurectoderm', 'Primitive Streak','Rostral neurectoderm']
for segment in segments:
    cell_data = prog[prog.obs['celltype'] == segment]
    random_index = np.random.choice(np.arange(0,np.shape(cell_data)[0]),100)
    cell_data = cell_data[random_index]
    downsampled_proge_data = downsampled_proge_data.concatenate(cell_data)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [105]:
# Starting data 
mature_data = mature[mature.obs['celltype'] == 'Forebrain/Midbrain/Hindbrain']
# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(mature_data)[0]),100)
downsampled_mature_data = mature_data[random_index]

segments = ['NMP', 'Neural crest','Spinal cord', 'Surface ectoderm']
for segment in segments:
    cell_data = mature[mature.obs['celltype'] == segment]
    random_index = np.random.choice(np.arange(0,np.shape(cell_data)[0]),100)
    cell_data = cell_data[random_index]
    downsampled_mature_data = downsampled_mature_data.concatenate(cell_data)
    

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [106]:
downsampled_ectoderm = downsampled_proge_data.concatenate(downsampled_mature_data)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [111]:
downsampled_mesoderm.obs["endo_trajectoryName"] = 0
downsampled_ectoderm.obs["endo_trajectoryName"] = 0

In [112]:
downsampled_mesoderm.write_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/Mesoderm_lineage_ds.h5ad")
downsampled_ectoderm.write_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/Ectoderm_lineage_ds.h5ad")